In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from sklearn.metrics import classification_report, mean_absolute_error, accuracy_score, f1_score
import csv
import os

In [2]:
batch_size = 32
num_epochs = 10
learning_rate = 1e-3
embed_dim = 128
hidden_dim = 256
num_layers = 2
dropout = 0.3

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda', index=0)

In [3]:
def read_file(filepath):
  return pd.read_csv(
      filepath,
      engine="python",
      sep = ",",
      escapechar='\\',
      quotechar='"',
      quoting=csv.QUOTE_MINIMAL,
      on_bad_lines='skip'
  )
col_emo = "Emotion"
col_emo_pol = "EmotionalPolarity"
col_emp = "Empathy"

train_df = read_file("/content/trac2_CONVT_train.csv")
test_df = read_file("/content/trac2_CONVT_test.csv")
dev_df = read_file("/content/trac2_CONVT_dev.csv")
train_df = train_df.query(f"{col_emo_pol} in [0, 1, 2]").reset_index(drop=True)
dev_df   = dev_df.query(f"{col_emo_pol} in [0, 1, 2]").reset_index(drop=True)

len(train_df), len(dev_df), len(test_df)



(10940, 952, 2316)

In [4]:
def tokenize(text):
  return text.lower().split()
pad_token = "<pad>"
unk_token = "<unk>"
def create_vocab(tokens):
  vocab = [pad_token, unk_token] + sorted(tokens)
  word2idx, idx2word = {},{}
  for i, word in enumerate(vocab):
    word2idx[word] = i
    idx2word[i] = word
  return word2idx, idx2word
all_texts = (
    train_df["text"].astype(str).tolist() +
    dev_df["text"].astype(str).tolist() +
    test_df["text"].astype(str).tolist()
)
tokens = set()
for text in all_texts:
  tokens.update(tokenize(text))
word2idx, idx2word = create_vocab(tokens)
vocab = word2idx
vocab_size = len(vocab)
print(vocab_size)

18365


In [5]:
from torch.utils.data import Dataset
import torch
from torch.nn.utils.rnn import pad_sequence

class c_dataset(Dataset):
  def __init__(self, texts, emotions, polarity, empathy, ids=None):
    self.texts = texts
    self.emotions = emotions
    self.polarity = polarity
    self.empathy = empathy
    self.ids = ids
  def __len__(self):
    return len(self.texts)
  def __getitem__(self, idx):
    tokens = [word2idx.get(word, word2idx[unk_token]) for word in tokenize(str(self.texts[idx]))]
    seq = torch.tensor(tokens, dtype=torch.long)
    item = {"seq": seq}
    if self.emotions is not None:
      item["emotion"] = torch.tensor(float(self.emotions[idx]), dtype=torch.float)
    if self.polarity is not None:
      item["polarity"] = torch.tensor(int(self.polarity[idx]), dtype=torch.long)
    if self.empathy is not None:
      item["empathy"] = torch.tensor(float(self.empathy[idx]), dtype=torch.float)
    if self.ids is not None:
      item["id"] = self.ids[idx]
    return item
  def padding_c(batch):
    seqs = [b["seq"] for b in batch]
    padded = pad_sequence(seqs, batch_first=True, padding_value=word2idx[pad_token])
    batch_out = {"seq": padded}
    for key in ["emotion", "polarity", "empathy", "id"]:
      if key in batch[0]:
        values = [b[key] for b in batch]
        if torch.is_tensor(values[0]):
          batch_out[key] = torch.stack(values)
        else:
          batch_out[key] = values
    return batch_out

In [6]:
class EmotionRNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout, num_classes=3):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True,
                        dropout=dropout if num_layers > 1 else 0)
    self.dropout = nn.Dropout(dropout)
    self.fc_emotion = nn.Linear(hidden_dim, 1)
    self.fc_polarity = nn.Linear(hidden_dim, num_classes)
    self.fc_empathy = nn.Linear(hidden_dim, 1)
  def forward(self, x):
    embedded = self.embedding(x)
    lstm_out, (hidden, _) = self.lstm(embedded)
    last_hidden = lstm_out[:, -1, :]
    h = self.dropout(last_hidden)
    emotion_out = self.fc_emotion(h)
    polarity_out = self.fc_polarity(h)
    empathy_out = self.fc_empathy(h)
    return emotion_out, polarity_out, empathy_out

In [7]:
def train_epoch(model, loader, optimizer, device):
  model.train()
  total_loss = 0.0
  mse = nn.MSELoss()
  ce = nn.CrossEntropyLoss()
  for batch in loader:
    seq = batch["seq"].to(device)
    emotion = batch["emotion"].to(device).unsqueeze(1)
    polarity = batch["polarity"].to(device)
    empathy = batch["empathy"].to(device).unsqueeze(1)
    optimizer.zero_grad()
    pred_emo, pred_pol, pred_emp = model(seq)
    loss = mse(pred_emo, emotion) + ce(pred_pol, polarity) + mse(pred_emp, empathy)
    loss.backward()
    optimizer.step()
    total_loss += float(loss.item()) * seq.size(0)
  return total_loss / len(loader.dataset)

def evaluate(model, loader, device):
  model.eval()
  all_true_emotion, all_pred_emotion = [], []
  all_true_polarity, all_pred_polarity = [], []
  all_true_empathy, all_pred_empathy = [], []
  with torch.no_grad():
    for batch in loader:
      seq = batch["seq"].to(device)
      emotion = batch["emotion"].numpy()
      polarity = batch["polarity"].numpy()
      empathy = batch["empathy"].numpy()
      pred_emo, pred_pol, pred_emp = model(seq)
      all_true_emotion.extend(pred_emo.cpu().squeeze().numpy())
      all_true_polarity.extend(pred_pol.argmax(dim=-1).cpu().numpy())
      all_true_empathy.extend(pred_emp.cpu().squeeze().numpy())
      all_pred_emotion.extend(emotion)
      all_pred_polarity.extend(polarity)
      all_pred_empathy.extend(empathy)
  mae_emotion = mean_absolute_error(all_pred_emotion, all_true_emotion)
  mae_empathy = mean_absolute_error(all_pred_empathy, all_true_empathy)
  acc_polarity = accuracy_score(all_pred_polarity, all_true_polarity)
  f1_polarity = f1_score(all_pred_polarity, all_true_polarity, average="macro")
  # print("\n--- DEV SET PERFORMANCE ---")
  # print(f"Emotion MAE: {mae_emotion:.4f}")
  # print(f"Empathy MAE: {mae_empathy:.4f}")
  # print(f"Polarity Accuracy: {acc_polarity:.4f}")
  # print(f"Polarity F1 (macro): {f1_polarity:.4f}")
  return mae_emotion, mae_empathy, acc_polarity, f1_polarity, classification_report(all_true_polarity, all_pred_polarity)



In [8]:
train_ds = c_dataset(train_df["text"], train_df[col_emo], train_df[col_emo_pol], train_df[col_emp])
dev_ds = c_dataset(dev_df["text"], dev_df[col_emo], dev_df[col_emo_pol], dev_df[col_emp])
test_ds = c_dataset(test_df["text"], None, None, None, ids=test_df["id"])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=c_dataset.padding_c)
dev_loader = DataLoader(dev_ds, batch_size=batch_size, shuffle=False, collate_fn=c_dataset.padding_c)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=c_dataset.padding_c)

In [9]:
model = EmotionRNN(vocab_size, embed_dim, hidden_dim, num_layers, dropout, num_classes=3).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, optimizer, DEVICE)
    mae_e, mae_emp, acc, f1, report = evaluate(model, dev_loader, DEVICE)
    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | MAE(Emotion)={mae_e:.4f} | MAE(Empathy)={mae_emp:.4f} | Acc(P)={acc:.4f} | F1(P)={f1:.4f}")

print("\nClassification report (dev, polarity):\n", report)

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 01 | train_loss=2.7902 | MAE(Emotion)=0.6143 | MAE(Empathy)=0.8883 | Acc(P)=0.4580 | F1(P)=0.2094


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 02 | train_loss=2.6801 | MAE(Emotion)=0.5867 | MAE(Empathy)=0.8909 | Acc(P)=0.3761 | F1(P)=0.1822


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 05 | train_loss=1.9396 | MAE(Emotion)=0.5518 | MAE(Empathy)=0.7842 | Acc(P)=0.5788 | F1(P)=0.4197


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 06 | train_loss=1.7793 | MAE(Emotion)=0.5491 | MAE(Empathy)=0.7949 | Acc(P)=0.5788 | F1(P)=0.4206


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 07 | train_loss=1.6416 | MAE(Emotion)=0.5232 | MAE(Empathy)=0.7944 | Acc(P)=0.5830 | F1(P)=0.4232
Epoch 08 | train_loss=1.5066 | MAE(Emotion)=0.5264 | MAE(Empathy)=0.7926 | Acc(P)=0.5788 | F1(P)=0.4670
Epoch 09 | train_loss=1.4254 | MAE(Emotion)=0.5682 | MAE(Empathy)=0.7971 | Acc(P)=0.5977 | F1(P)=0.4960
Epoch 10 | train_loss=1.3425 | MAE(Emotion)=0.5457 | MAE(Empathy)=0.7822 | Acc(P)=0.5746 | F1(P)=0.4725

Classification report (dev, polarity):
               precision    recall  f1-score   support

           0       0.11      0.63      0.18        27
           1       0.61      0.61      0.61       440
           2       0.73      0.54      0.62       485

    accuracy                           0.57       952
   macro avg       0.48      0.59      0.47       952
weighted avg       0.66      0.57      0.60       952



In [10]:
# Predict on test and save CSV
model.eval()
all_rows = []
with torch.no_grad():
    for batch in test_loader:
        seq = batch["seq"].to(DEVICE)
        ids = batch["id"]
        pred_emo, pred_pol, pred_emp = model(seq)
        emo = pred_emo.cpu().squeeze().numpy()
        pol = pred_pol.argmax(dim=-1).cpu().numpy()
        emp = pred_emp.cpu().squeeze().numpy()
        for i in range(len(ids)):
            all_rows.append({
                "id": int(ids[i]),
                "Emotion": float(emo[i]),
                "EmotionalPolarity": int(pol[i]),
                "Empathy": float(emp[i]),
            })

pred_df = pd.DataFrame(all_rows, columns=["id","Emotion","EmotionalPolarity","Empathy"])
out_csv = os.path.join("/content/", "predictions_ann.csv")
pred_df.to_csv(out_csv, index=False)
print(pred_df["EmotionalPolarity"].value_counts())


EmotionalPolarity
2    1240
1    1017
0      59
Name: count, dtype: int64
